<a href="https://colab.research.google.com/github/Bat4E/Csci166-RL-DQN-Final-Project/blob/main/notebooks/Variant_Donkey_Kong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test 3 --- Final Test for 06.11.2025

In [ ]:
!pip install gymnasium[atari,accept-rom-license]
!pip install autorom
!pip install stable-baselines3

In [ ]:
!AutoROM --accept-license

AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	/usr/local/lib/python3.12/dist-packages/AutoROM/roms

Existing ROMs will be overwritten.


# Install the Gym

In [ ]:
import ale_py
import gymnasium as gym

# Configure the model save drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
# Define directories
save_dir_drive = "/content/drive/MyDrive/PUBLIC/Models"
save_dir_local = "saved_models"
video_dir_drive = "/content/drive/MyDrive/PUBLIC/Videos" # <--- ADDED THIS LINE

# Create both directories if they don't exist
os.makedirs(save_dir_drive, exist_ok=True)
os.makedirs(save_dir_local, exist_ok=True)
os.makedirs(video_dir_drive, exist_ok=True) # <--- ADDED THIS LINE

# Now Model

In [ ]:
from dataclasses import dataclass
import argparse
import time
from datetime import datetime
import numpy as np
import collections
import typing as tt

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.tensorboard.writer import SummaryWriter

In [ ]:
#DuelingDQN_model
# --- Dueling DQN Network ---
class DuelingDQN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(DuelingDQN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4), nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.ReLU(),
            nn.Flatten()
        )
        conv_out_size = self._get_conv_out(input_shape)

        self.fc_adv = nn.Sequential(
            nn.Linear(conv_out_size, 512), nn.ReLU(),
            nn.Linear(512, n_actions)
        )
        self.fc_val = nn.Sequential(
            nn.Linear(conv_out_size, 512), nn.ReLU(),
            nn.Linear(512, 1)
        )

    def _get_conv_out(self, shape):
        o = torch.zeros(1, *shape)
        return int(np.prod(self.conv(o).size()))

    def forward(self, x):
        x = self.conv(x)
        adv = self.fc_adv(x)
        val = self.fc_val(x)
        return val + adv - adv.mean(dim=1, keepdim=True)



In [ ]:
# wrappers
#wrappers

import collections
import numpy as np
import torch
import random
import typing as tt

from gymnasium import spaces
from gymnasium.wrappers import RecordVideo  # added to Record Videos
from stable_baselines3.common import atari_wrappers
import gymnasium as gym


class ImageToPyTorch(gym.ObservationWrapper):
    """
    ImageToPyTorch: Reorders image dimensions from (H, W, C) to (C, H, W)
    for compatibility with PyTorch convolutional layers.
    """
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        obs = self.observation_space
        assert isinstance(obs, gym.spaces.Box)
        assert len(obs.shape) == 3
        new_shape = (obs.shape[-1], obs.shape[0], obs.shape[1])
        self.observation_space = gym.spaces.Box(
            low=obs.low.min(), high=obs.high.max(),
            shape=new_shape, dtype=obs.dtype)

    def observation(self, observation):
        return np.moveaxis(observation, 2, 0)


class BufferWrapper(gym.ObservationWrapper):
    """
    BufferWrapper: Maintains a rolling window of the last `n_steps` frames
    to give the agent a sense of temporal context.
    """
    def __init__(self, env, n_steps):
        super(BufferWrapper, self).__init__(env)
        obs = env.observation_space
        assert isinstance(obs, spaces.Box)
        new_obs = gym.spaces.Box(
            obs.low.repeat(n_steps, axis=0), obs.high.repeat(n_steps, axis=0),
            dtype=obs.dtype)
        self.observation_space = new_obs
        self.buffer = collections.deque(maxlen=n_steps)

    def reset(self, *, seed: tt.Optional[int] = None, options: tt.Optional[dict[str, tt.Any]] = None):
        for _ in range(self.buffer.maxlen):
            self.buffer.append(np.zeros_like(self.env.observation_space.low))
        obs, extra = self.env.reset()
        return self.observation(obs), extra

    def observation(self, observation: np.ndarray) -> np.ndarray:
        self.buffer.append(observation)
        return np.concatenate(self.buffer)


def make_env(env_name: str, n_steps=4, render_mode=None,
             record_video=False, video_folder=None,
             episode_trigger=None, video_prefix=None):
    print(f"Creating environment {env_name}")
    env = gym.make(env_name, render_mode=render_mode)
    env = atari_wrappers.AtariWrapper(env, clip_reward=False, noop_max=0)
    if record_video:
        env = RecordVideo(
            env,
            video_folder=video_folder,
            episode_trigger=episode_trigger if episode_trigger is not None else lambda x: x == 0,
            name_prefix=video_prefix if video_prefix else f"{env_name.replace('/', '_')}"
        )
    env = ImageToPyTorch(env)
    env = BufferWrapper(env, n_steps=n_steps)
    return env


# --- Helper function for fixed-length video recording (~10s clips) ---
def record_fixed_steps(env_name, video_folder, prefix, steps=600, epsilon=1.0, net=None):
    """
    Record a short clip (~10 seconds) of agent gameplay.
    - env_name: Atari env string
    - video_folder: where to save video
    - prefix: filename prefix
    - steps: number of frames (~600 \u2248 10s at 60fps)
    - epsilon: exploration rate (1.0 = random, 0.0 = greedy)
    - net: trained network (None = random actions only)
    """
    # Modified: Use make_env to ensure proper preprocessing
    env = make_env(
        env_name=env_name,
        render_mode="rgb_array",
        record_video=True,
        video_folder=video_folder,
        episode_trigger=lambda ep: ep == 0,
        video_prefix=prefix
    )
    state, _ = env.reset()
    total_reward = 0
    for step in range(steps):
        if random.random() < epsilon or net is None:
            action = env.action_space.sample()
        else:
            with torch.no_grad():
                s = torch.tensor(state).unsqueeze(0).to(torch.float32)
                action = net(s).argmax().item()
        state, reward, terminated, truncated, _ = env.step(action)
        total_reward += reward
        if terminated or truncated:
            state, _ = env.reset()
    env.close()
    print(f"{prefix} video recorded, reward {total_reward}")

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
# Base Configuration
DEFAULT_ENV_NAME = "ALE/DonkeyKong-v5" # Changed to Donkey Kong
MEAN_REWARD_BOUND = 500 # Raised from 5 to 500 for meaningful training

GAMMA = 0.99
BATCH_SIZE = 16
REPLAY_SIZE = 5000
LEARNING_RATE = 1e-4
SYNC_TARGET_FRAMES = 500
REPLAY_START_SIZE = 1000

SAVE_EPSILON = 1.0  # Only save if at least this much better
EPSILON_DECAY_LAST_FRAME = 10000
EPSILON_START = 1.0
EPSILON_FINAL = 0.02

# Threshold for recording "learned" video
LEARNED_VIDEO_REWARD_THRESHOLD = 500

# Tuple of tensors returned from a sampled minibatch in replay buffer
State = np.ndarray
Action = int
BatchTensors = tt.Tuple[
    torch.ByteTensor,           # current state
    torch.LongTensor,           # actions
    torch.Tensor,               # rewards
    torch.BoolTensor,           # done || trunc
    torch.ByteTensor            # next state
]

In [ ]:
# ⚙️ Fast Training Config for Quick Test Run
MEAN_REWARD_BOUND = 500 # raised to 500
REPLAY_START_SIZE = 1000
EPSILON_DECAY_LAST_FRAME = 10_000
SYNC_TARGET_FRAMES = 500

REPLAY_SIZE = 5000  # optional, uncommented
BATCH_SIZE = 16     # optional, uncommented

In [ ]:
import os
from pathlib import Path

# Define directories
save_dir_drive = "/content/drive/MyDrive/PUBLIC/Models"
save_dir_local = "saved_models"

# Create both directories if they don't exist
os.makedirs(save_dir_drive, exist_ok=True)
os.makedirs(save_dir_local, exist_ok=True)

# Safe model filename
env_name = DEFAULT_ENV_NAME
safe_env_name = env_name.replace("/", "_")

In [ ]:
# --- Prioritized Replay Buffer ---
class PrioritizedReplayBuffer:
    def __init__(self, capacity, alpha=0.6):
        self.capacity = capacity
        self.alpha = alpha
        self.buffer = []
        self.pos = 0
        self.priorities = np.zeros((capacity,), dtype=np.float32)

    def push(self, experience):
        max_prio = self.priorities.max() if self.buffer else 1.0
        if len(self.buffer) < self.capacity:
            self.buffer.append(experience)
        else:
            self.buffer[self.pos] = experience
        self.priorities[self.pos] = max_prio
        self.pos = (self.pos + 1) % self.capacity

    def sample(self, batch_size, beta=0.4):
        if len(self.buffer) == self.capacity:
            prios = self.priorities
        else:
            prios = self.priorities[:self.pos]
        probs = prios ** self.alpha
        probs /= probs.sum()

        idxs = np.random.choice(len(self.buffer), batch_size, p=probs)
        samples = [self.buffer[idx] for idx in idxs]

        total = len(self.buffer)
        weights = (total * probs[idxs]) ** (-beta)
        weights /= weights.max()
        return [(*samples[i], weights[i], idxs[i]) for i in range(batch_size)]

    def update_priorities(self, idxs, prios):
        for idx, prio in zip(idxs, prios):
            self.priorities[idx] = prio + 1e-5

    def __len__(self):
        return len(self.buffer)


In [ ]:
class Agent:
    def __init__(self, env, exp_buffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self.state = None
        self._reset()

    def _reset(self):
        self.state, _ = self.env.reset()
        self.total_reward = 0.0

    @torch.no_grad()
    def play_step(self, net, device, epsilon: float = 0.0):
        done_reward = None

        # ε-greedy action selection
        if np.random.random() < epsilon:
            action = self.env.action_space.sample()
        else:
            # Cast to float32 and normalize to [0,1]
            state_v = torch.as_tensor(self.state, dtype=torch.float32).to(device).unsqueeze(0)
            state_v /= 255.0
            q_vals_v = net(state_v)
            _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())

        # Step in environment
        new_state, reward, is_done, is_tr, _ = self.env.step(action)
        self.total_reward += reward

        # Store experience in PER buffer
        exp = (self.state, action, float(reward), is_done or is_tr, new_state)
        self.exp_buffer.push(exp)

        self.state = new_state
        if is_done or is_tr:
            done_reward = self.total_reward
            self._reset()
        return done_reward


In [ ]:
# --- Double DQN Loss ---
def calc_loss(batch, net, tgt_net, device, gamma=GAMMA):
    states, actions, rewards, dones, next_states, weights, idxs = zip(*batch)

    states_v = torch.tensor(np.array(states), dtype=torch.float32).to(device) / 255.0
    actions_v = torch.tensor(actions).to(torch.int64).unsqueeze(-1).to(device)
    rewards_v = torch.tensor(rewards, dtype=torch.float32).to(device)
    dones_t = torch.BoolTensor(dones).to(device)
    next_states_v = torch.tensor(np.array(next_states), dtype=torch.float32).to(device) / 255.0
    weights_v = torch.tensor(weights, dtype=torch.float32).to(device)

    state_action_values = net(states_v).gather(1, actions_v).squeeze(-1)

    # Double DQN
    next_state_actions = net(next_states_v).argmax(dim=1, keepdim=True)
    next_state_values = tgt_net(next_states_v).gather(1, next_state_actions).squeeze(-1)
    next_state_values[dones_t] = 0.0

    expected_values = rewards_v + gamma * next_state_values
    loss = (state_action_values - expected_values.detach()).pow(2) * weights_v

    # detach before numpy
    td_errors = (state_action_values.detach() - expected_values.detach()).abs().cpu().numpy()

    return loss.mean(), td_errors, idxs


**Main Training Loop**

In [ ]:
# Assuming 'model_comment', 'safe_env_name', 'DEFAULT_ENV_NAME',
# 'save_dir_drive', 'save_dir_local' are already defined earlier in your script.
# Also ensure 'video_dir_drive' is defined and its directory created.
# E.g., add this where save_dir_drive and save_dir_local are defined:
# video_dir_drive = "/content/drive/MyDrive/PUBLIC/Videos"
# os.makedirs(video_dir_drive, exist_ok=True)

# --- Longer Video Recording (~30s = 1800 frames) ---
def record_fixed_steps(env_name, video_folder, prefix, steps=1800, epsilon=1.0, net=None, device="cpu"):
    env = make_env(env_name, n_steps=4, render_mode="rgb_array",
                   record_video=True, video_folder=video_folder,
                   episode_trigger=lambda ep: ep == 0,
                   video_prefix=prefix)
    state, _ = env.reset()
    total_reward = 0
    for step in range(steps):
        if random.random() < epsilon or net is None:
            action = env.action_space.sample()
        else:
            with torch.no_grad():
                s = torch.tensor(np.array(state)).unsqueeze(0).to(torch.float32).to(device)
                action = net(s).argmax().item()
        state, reward, terminated, truncated, _ = env.step(action)
        total_reward += reward
        if terminated or truncated:
            state, _ = env.reset()
    env.close()
    print(f"{prefix} video recorded, reward {total_reward}")


# --- Training Loop (Variant Agent) ---
model_comment = "variant_double_dueling_per"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("\n--- Recording Early/Randomish Video (Variant) ---")
record_fixed_steps(DEFAULT_ENV_NAME, video_dir_drive, "variant_start", steps=1800, epsilon=1.0, device=device)

env = make_env(env_name=DEFAULT_ENV_NAME)
print(f"\nObservation space shape: {env.observation_space.shape}")
print(f"Number of actions: {env.action_space.n}")

net = DuelingDQN(env.observation_space.shape, env.action_space.n).to(device)
tgt_net = DuelingDQN(env.observation_space.shape, env.action_space.n).to(device)
writer = SummaryWriter(comment=f"-{DEFAULT_ENV_NAME.replace('/', '_')}-{model_comment}")
print(net)

buffer = PrioritizedReplayBuffer(REPLAY_SIZE)
agent = Agent(env, buffer)
epsilon = EPSILON_START

optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
total_rewards = []
frame_idx = 0
ts_frame = 0
ts = time.time()
best_m_reward = None
learned_video_recorded = False
LEARNED_VIDEO_REWARD_THRESHOLD = 500

start_time = time.time()
while True:
    frame_idx += 1
    epsilon = max(EPSILON_FINAL, EPSILON_START - frame_idx / EPSILON_DECAY_LAST_FRAME)

    reward = agent.play_step(net, device, epsilon)
    if reward is not None:
        total_rewards.append(reward)
        speed = (frame_idx - ts_frame) / (time.time() - ts)
        elapsed = time.time() - start_time
        ts_frame = frame_idx
        ts = time.time()
        m_reward = np.mean(total_rewards[-100:])

        writer.add_scalar("epsilon", epsilon, frame_idx)
        writer.add_scalar("speed", speed, frame_idx)
        writer.add_scalar("reward_100", m_reward, frame_idx)
        writer.add_scalar("reward", reward, frame_idx)

        print(f"[DEBUG] Episode finished with reward {reward:.2f}, mean_100 {m_reward:.2f}")

        if best_m_reward is None or m_reward > best_m_reward + SAVE_EPSILON:
            timestamp = datetime.now().strftime("%Y%m%d-%H%M")
            model_filename = f"{safe_env_name}-best_{int(m_reward)}-{timestamp}-{model_comment}.dat"
            model_path_drive = os.path.join(save_dir_drive, model_filename)
            model_path_local = os.path.join(save_dir_local, model_filename)
            torch.save(net.state_dict(), model_path_drive)
            torch.save(net.state_dict(), model_path_local)
            print(f"💾 Model saved to:\n - Google Drive: {model_path_drive}\n - Local: {model_path_local}")
            best_m_reward = m_reward

            if not learned_video_recorded and m_reward > LEARNED_VIDEO_REWARD_THRESHOLD:
                print("\n--- Recording Later/Learned Video (Variant) ---")
                record_fixed_steps(DEFAULT_ENV_NAME, video_dir_drive, "variant_end", steps=1800, epsilon=0.0, net=net, device=device)
                learned_video_recorded = True
                print("Later/Learned video recorded.")

    # --- Midpoint clip ---
    if frame_idx == 250000:
        print("\n--- Recording Midpoint Video (Variant) ---")
        record_fixed_steps(DEFAULT_ENV_NAME, video_dir_drive, "variant_mid", steps=1800, epsilon=0.1, net=net, device=device)
        print("Midpoint video recorded.")

    if frame_idx > 500000 and m_reward > MEAN_REWARD_BOUND:
        print("Solved in %d frames!" % frame_idx)
        break

    if frame_idx % 2000 == 0:
        print(f"[DEBUG] Buffer size: {len(buffer)} / {REPLAY_START_SIZE}")

    if len(buffer) < REPLAY_START_SIZE:
        continue

    if frame_idx % SYNC_TARGET_FRAMES == 0:
        tgt_net.load_state_dict(net.state_dict())
        print(f"[DEBUG] Target network synced at frame {frame_idx}")

    optimizer.zero_grad()
    batch = buffer.sample(BATCH_SIZE)
    loss_t, prios, idxs = calc_loss(batch, net, tgt_net, device)
    loss_t.backward()
    optimizer.step()

    # Update PER priorities
    buffer.update_priorities(idxs, prios)

    if frame_idx % 1000 == 0:
        print(f"[DEBUG] Loss at frame {frame_idx}: {loss_t.item():.6f}")

    if frame_idx % 10000 == 0:
        sample_batch = buffer.sample(BATCH_SIZE)
        actions = [exp[1] for exp in sample_batch]  # action is 2nd element
        unique, counts = np.unique(actions, return_counts=True)
        print(f"[DEBUG] Action distribution at {frame_idx}: {dict(zip(unique, counts))}")

env.close()
writer.close()




--- Recording Early/Randomish Video (Variant) ---
Creating environment ALE/DonkeyKong-v5


/usr/local/lib/python3.12/dist-packages/gymnasium/wrappers/rendering.py:293: UserWarning: WARN: Overwriting existing videos at /content/drive/MyDrive/PUBLIC/Videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


variant_start video recorded, reward 1300.0
Creating environment ALE/DonkeyKong-v5

Observation space shape: (4, 84, 84)
Number of actions: 18


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


DuelingDQN(
  (conv): Sequential(
    (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
    (6): Flatten(start_dim=1, end_dim=-1)
  )
  (fc_adv): Sequential(
    (0): Linear(in_features=3136, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=18, bias=True)
  )
  (fc_val): Sequential(
    (0): Linear(in_features=3136, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=1, bias=True)
  )
)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


[DEBUG] Episode finished with reward 0.00, mean_100 0.00
💾 Model saved to:
 - Google Drive: /content/drive/MyDrive/PUBLIC/Models/ALE_DonkeyKong-v5-best_0-20251108-2110-variant_double_dueling_per.dat
 - Local: saved_models/ALE_DonkeyKong-v5-best_0-20251108-2110-variant_double_dueling_per.dat
[DEBUG] Episode finished with reward 0.00, mean_100 0.00
[DEBUG] Episode finished with reward 0.00, mean_100 0.00
[DEBUG] Episode finished with reward 0.00, mean_100 0.00
[DEBUG] Episode finished with reward 0.00, mean_100 0.00
[DEBUG] Episode finished with reward 0.00, mean_100 0.00
[DEBUG] Episode finished with reward 0.00, mean_100 0.00
[DEBUG] Episode finished with reward 0.00, mean_100 0.00
[DEBUG] Episode finished with reward 100.00, mean_100 11.11
💾 Model saved to:
 - Google Drive: /content/drive/MyDrive/PUBLIC/Models/ALE_DonkeyKong-v5-best_11-20251108-2110-variant_double_dueling_per.dat
 - Local: saved_models/ALE_DonkeyKong-v5-best_11-20251108-2110-variant_double_dueling_per.dat
[DEBUG] Epis

/usr/local/lib/python3.12/dist-packages/gymnasium/wrappers/rendering.py:293: UserWarning: WARN: Overwriting existing videos at /content/drive/MyDrive/PUBLIC/Videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


variant_end video recorded, reward 8200.0
Later/Learned video recorded.
[DEBUG] Buffer size: 5000 / 1000
[DEBUG] Target network synced at frame 32000
[DEBUG] Loss at frame 32000: 158.425873
[DEBUG] Episode finished with reward 800.00, mean_100 516.00
💾 Model saved to:
 - Google Drive: /content/drive/MyDrive/PUBLIC/Models/ALE_DonkeyKong-v5-best_516-20251108-2116-variant_double_dueling_per.dat
 - Local: saved_models/ALE_DonkeyKong-v5-best_516-20251108-2116-variant_double_dueling_per.dat
[DEBUG] Target network synced at frame 32500
[DEBUG] Episode finished with reward 2000.00, mean_100 536.00
💾 Model saved to:
 - Google Drive: /content/drive/MyDrive/PUBLIC/Models/ALE_DonkeyKong-v5-best_536-20251108-2116-variant_double_dueling_per.dat
 - Local: saved_models/ALE_DonkeyKong-v5-best_536-20251108-2116-variant_double_dueling_per.dat
[DEBUG] Episode finished with reward 700.00, mean_100 542.00
💾 Model saved to:
 - Google Drive: /content/drive/MyDrive/PUBLIC/Models/ALE_DonkeyKong-v5-best_542-2025

KeyboardInterrupt: 